In [18]:
import sys
sys.path.append('/workspaces/tools-jupyter-notebooks/protos_bin')
import stratum_rpc_pb2 as rpc

We have really big proto files that need to be sent to the backend, but they are too big. The problem is that there were a lot of rig connect / disconnects because of enforced KYC for miners. Non KYC miners were kicked off and they were trying to reconnect immediatelly, therefore bloating and expanding proto files with a lot of 0 amount rig entries.

Plan:
1. Sum all mining amounts together
2. Remove all rigs with 0 mining amount from proto
3. Check again if all mining amounts match with this cleared proto

Calculate cumulative amount prior proto change

In [19]:
rig_request = rpc.RpcRigRequest()
with open('/workspaces/tools-jupyter-notebooks/data/1731947473.proto', "rb") as f:
  rig_request.ParseFromString(f.read())

cumulative_amount_paid = 0
for rig_data in rig_request.rigs:
    cumulative_amount_paid += rig_data.amountPaid
print(f"Total amount paid: {cumulative_amount_paid}")

Total amount paid: 64935


Modify proto and save

In [20]:
to_remove = []
for i, rig_data in enumerate(rig_request.rigs):
    if rig_data.amountPaid == 0:
        to_remove.append(i)
for i in reversed(to_remove):
    del rig_request.rigs[i]
with open('/workspaces/tools-jupyter-notebooks/data/1731947473_cleaned.proto', "wb") as f:
  f.write(rig_request.SerializeToString())

Compare cumulativ amounts paid

In [21]:
rig_request_orig = rpc.RpcRigRequest()
with open('/workspaces/tools-jupyter-notebooks/data/1731947473.proto', "rb") as f:
  rig_request_orig.ParseFromString(f.read())

rig_request_cleaned = rpc.RpcRigRequest()
with open('/workspaces/tools-jupyter-notebooks/data/1731947473_cleaned.proto', "rb") as f:
  rig_request_cleaned.ParseFromString(f.read())

cumulative_amount_paid_orig = 0
for rig_data in rig_request_orig.rigs:
    cumulative_amount_paid_orig += rig_data.amountPaid

cumulative_amount_paid_cleaned = 0
for rig_data in rig_request_cleaned.rigs:
    cumulative_amount_paid_cleaned += rig_data.amountPaid

print("Original cumulative amount paid: ", cumulative_amount_paid_orig)
print("Cleaned cumulative amount paid: ", cumulative_amount_paid_cleaned)

Original cumulative amount paid:  64935
Cleaned cumulative amount paid:  64935
